## Ildebrando Magnani

im975

Homework #5

This program shows the downward bias in the OLS estimate of α in the AR(1) process. 

The estimation is performed for different pairs of "true" alphas and sample sizes.

I've been having problems with Julia, as the confusion of this code shows. 







We generate a function that runs AR(1) simulation. 
It returns 2 arrays with "lagged" and "non-lagged" observations.

In [47]:
using Distributions: Normal
srand(15) #seed selection


function ar1_gen(number_of_obs::Int64, alpha::Float64, beta::Float64, s::Float64)
    
    x = zeros(simulation_length)
    x[1] = beta / (1 - alpha)
    w = rand(Normal(0.0, 1.0))
    
    for t in 1:number_of_obs
        x[t+1] = beta + alpha * x[t] + s * w[t+1]
    end
    
    x_lagged_obs = x[1:simulation_length-1]
    x_obs = x[2:simulation_length]
    return x_lagged_obs, x_obs

end




ar1_gen (generic function with 2 methods)

To make a cleaner code, I try to break up the estimation of alpha and the computation of the biases into 2 different functions.

Just below we have the function that estimates the alpha parameter.

In [48]:
function alpha_estimation(repetition::Int64)
    
    for i in 1:repetition
        xlag, x = ar1_gen(sim_N[n], a, 1, 1)
        xlag_2 = xlag ^ 2
        x_2 = x^2
        xlag_times_x = xlag * x
        xlag_times_x_bar = mean(xlag_times_x)
        xlag_bar = mean(xlag)
        x_bar = mean(xbar)
        xlag_2_bar = mean(xlag_2)
        xlag_bar_2 = xlag_bar ^ 2
        cov = xlag_times_x_bar - (xlag_bar * x_bar)
        var = var = xlag_2_bar - xlag_bar_2
        alpha_est = cov / var
        append!(alpha_estimates, alpha_est)
    end
    return alpha_estimates

end
    

alpha_estimation (generic function with 2 methods)

I now define 2 arrays that contain different "true alpha" values and different numbers of AR(1) observations.

Then, I create a function that should compute the biases for each (alpha_vals, sim_N) touple.

In [49]:
alpha_vals = linspace(0.5, 0.9, 5)
sim_N = linspace(50, 500, 10)
biases = zeros(50)
experiment_repetition = 10000

function bias_compute(N::Int64, alphas::Float64)
    
@time   for x in 1:length(alpha_vals)
        a = x 
        for n in 1:length(sim_N)
            alpha_estimates = zeros(Float64)
            alpha_estimates = alpha_estimation(experiment_repetition)  
            bias = mean(alpha_estimates) - a 
            append!(biases, bias)
        end
    end
    return biases

end


bias_compute (generic function with 2 methods)

In [50]:
bias_compute(sim_N, alpha_vals)

LoadError: LoadError: MethodError: `bias_compute` has no method matching bias_compute(::LinSpace{Float64}, ::LinSpace{Float64})
while loading In[50], in expression starting on line 1

I replicate the code from last homework to plot the results

In [51]:
using PyPlot

# "Unpack" the values of the biases for different values of true alpha
a5 = biases[1:10]
a6 = biases[11:20]
a7 = biases[21:30]
a8 = biases[31:40]
a9 = biases[41:50]

# Plot the results
fig, ax = plt.subplots(figsize=(8, 6))
x = np.linspace(50, 500, 10)
plt.plot(x, a5, color = 'red', linewidth=1, label = 'alpha = 0.5', alpha=0.8)
plt.plot(x, a6, color = 'green', linewidth=1, label = 'alpha = 0.6', alpha=0.8)
plt.plot(x, a7, color = 'yellow', linewidth=1, label = 'alpha = 0.7', alpha=0.8)
plt.plot(x, a8, color = 'black', linewidth=1, label = 'alpha = 0.8', alpha=0.8)
plt.plot(x, a9, color = 'brown', linewidth=1, label = 'alpha = 0.9', alpha=0.8)
plt.xlabel('Sample Size')
plt.ylabel('Bias of OLS estimator alpha')
ax.legend(loc='lower right')
plt.show()

LoadError: LoadError: type PyObject has no field subplots
while loading In[51], in expression starting on line 11

## I apologize for the mess
